##2025년 금융데이터 분석사   09.26

### 문제_01 데이터 수집 및 시각화

## 문제 개요

한국은행 ECOS API에서 인터넷뱅킹 관련 통계(stat_code=631Y001)의 자금이체(금액) 항목(item_code1=H2200)을 연간 단위로 내려받아 전처리(결측치 처리 포함)하고, 지정된 두 가지 시각화도구를 생성하시오.

## 1) 데이터 수집 정보

- 통계표 코드 (stat_code): 631Y001
- 주기 (frequency): A (연간, YYYY)
- 세부항목 (item_code1): H2200 (자금이체(금액), 단위: 억원)
- 분석 기간: 2000 (2000년) 부터 2023 (2023년) 까지

요청 URL 포맷(예시):

https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/9999/631Y001/A/2000/2023/H2200


## 2) 전처리 요구 사항

API로 받아온 원시 데이터에서 TIME 값을 연도(year, 정수)와 date (YYYY-01-01 형식의 datetime)로 변환하고, date를 인덱스로 설정하여 오름차순 정렬하시오.
설명: TIME은 연도 표기(예: 2009)로 들어오며, 날짜 인덱스를 만들어 시계열 연산에 사용해야 합니다.

값 컬럼(숫자값)을 자동으로 식별하여 내부의 콤마(,)를 제거한 뒤 숫자형으로 변환하시오. 변환 불가 항목은 NaN으로 처리하시오.
설명: 컬럼명이 항상 같지 않으므로 "DATA"와 "VALUE"가 섞인 이름을 우선 찾고, 없으면 숫자로 변환 가능한 첫 컬럼을 사용하세요.

결측치 처리: 연속성 유지를 위해 value의 결측은 앞채우기(ffill) 후 뒤채우기(bfill) 방식으로 채워 value_filled 컬럼을 만들고, 결측 처리 전/후의 결측 개수를 출력하시오.
설명: 결측 처리 전/후 코드는 반드시 추가하지만 만약 샘플 데이터에 결측치가 존재하지 않는다면 그 값을 같을 수 있습니다.

### [전처리 작성 중 새롭게 추가될 컬럼 정보]

- ma_3 : 3년 이동평균(시계열 특성상 min_periods=1로 계산)
- rolling_std_3 : 3년 이동표준편차(빈값은 0으로 대체 가능)
- lag_3 : 3년 전 값(shift(3))
- value_norm : Min–Max 정규화(0~1 범위)
각각의 컬럼이 생성되었음을 확인(예: 상위 5행 출력)하시오.


## 3) 시각화 요구 사항

3-1) 그래프 A : Area plot

 내용: value_filled를 면(영역)으로 채우고 그 위에 ma_3(3-year moving average) 선을 겹쳐 표시하시오.

시각 요소: 제목 "Area plot: Value (filled) + 3-year MA", x축 "Year", y축 "Amount (unit as provided)", 범례 표시, 격자 표시.

x축 데이터는 연도(정수)로 표시하여 가독성 확보.

3-2) 그래프 B — Bar + secondary-line

내용: 왼쪽 축에 value_filled 막대그래프(bar), 오른쪽(보조 y축)에 rolling_std_3를 선(line)으로 표시하시오.

시각 요소: 제목 "Bar (value) + Line (3yr stddev)", 왼쪽 y축 레이블 "Value (filled)", 오른쪽 y축 레이블 "Rolling Std (3yr)", 범례를 각 축에 적절히 표시.

보조 y축을 사용할 때는 각 축 단위를 명확히 표기하시오.


## 제출 파일
1) 전처리된 Dataframe 저장
   -- 최종 전처리된 DataFrame을 금융분석가_문제1.csv 이름으로 저장하시오 (인코딩 utf-8-sig 권장)

2) 작성된 금융분석가_문제1.ipynb

2개 파일을 로컬 PC로 다운로드하여 제출합니다.

### [참고]

<img src="https://imgur.com/mG6UmOH.png" width="80%">

<img src="https://imgur.com/vpBvq3u.png" width="80%">

In [ ]:

import requests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,5)
sns.set_style('whitegrid')

